# Podcast Creator - Google Colab Notebook

This notebook creates a podcast from Wikipedia articles, Perplexity searches, or uploaded content.

## Step 1: Install Dependencies

In [7]:
# ============================================
# STEP 1: Install all required dependencies
# IMPORTANT: Run this cell FIRST before any other cells!
# ============================================

# Install ffmpeg FIRST (required for pydub audio combining)
import subprocess
import sys

# Check if we're in Colab
try:
    import google.colab
    IN_COLAB = True
    print("📍 Running in Google Colab")
    # Install ffmpeg in Colab
    !apt-get update -qq
    !apt-get install -y ffmpeg -qq
    print("✓ ffmpeg installed successfully")
    from google.colab import userdata
    userdata.get('GROQ_API_KEY')
except ImportError:
    IN_COLAB = False
    print("📍 Running locally - please ensure ffmpeg is installed:")
    print("   Mac: brew install ffmpeg")
    print("   Windows: Download from ffmpeg.org")
    print("   Linux: sudo apt-get install ffmpeg")

# Install Python packages
%pip install -q groq wikipedia requests edge-tts pydub nest_asyncio

print("\n✓ All packages installed!")
print("\n📢 Edge TTS provides FREE high-quality Microsoft Neural TTS voices:")
print("   - Indian English: Neerja (F), Prabhat (M)")
print("   - Hindi: Swara (F), Madhur (M)")
print("   - Tamil: Pallavi (F), Valluvar (M)")
print("   - Telugu: Shruti (F), Mohan (M)")

📍 Running in Google Colab
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
✓ ffmpeg installed successfully

✓ All packages installed!

📢 Edge TTS provides FREE high-quality Microsoft Neural TTS voices:
   - Indian English: Neerja (F), Prabhat (M)
   - Hindi: Swara (F), Madhur (M)
   - Tamil: Pallavi (F), Valluvar (M)
   - Telugu: Shruti (F), Mohan (M)


## Step 2: Import Libraries and Setup

In [8]:
# ============================================
# STEP 2: Import Libraries and Setup
# ============================================

import json
import os
import asyncio
import warnings

# Suppress pydub ffmpeg warnings
warnings.filterwarnings("ignore", message="Couldn't find ffmpeg")
warnings.filterwarnings("ignore", message="Couldn't find ffprobe")

from groq import Groq
import wikipedia

# Edge TTS for high-quality neural voices
import edge_tts
import nest_asyncio
nest_asyncio.apply()  # Allow nested event loops (needed for Jupyter/Colab)

# For audio playback in notebooks (works without ffmpeg)
try:
    from IPython.display import Audio, display
    AUDIO_DISPLAY_AVAILABLE = True
except ImportError:
    AUDIO_DISPLAY_AVAILABLE = False
    print("⚠ IPython.display not available - audio playback disabled")

# Try pydub for audio combining (optional - fallback available)
PYDUB_AVAILABLE = False
try:
    # Suppress warnings before importing pydub
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        from pydub import AudioSegment
        # Test if ffmpeg is actually available
        try:
            AudioSegment.silent(duration=100)
            PYDUB_AVAILABLE = True
            print("✓ pydub + ffmpeg available for high-quality audio combining")
        except:
            print("ℹ pydub available but ffmpeg not found - using fallback audio combining")
except ImportError:
    print("ℹ pydub not installed - using fallback audio combining")

# Try to import google.colab (only available in Colab)
try:
    from google.colab import files
    from google.colab import userdata # Added for Colab secrets
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
    print("Note: Not running in Colab. File upload will use standard file input.")

# API Keys (Groq for script generation)
if IN_COLAB:
    GROQ_API_KEY = userdata.get('GROQ_API_KEY') or os.environ.get("GROQ_API_KEY") or input("🔑 Enter your Groq API Key: ")
else:
    GROQ_API_KEY = os.environ.get("GROQ_API_KEY") or input("🔑 Enter your Groq API Key: ")

# Initialize Groq client
groq_client = Groq(api_key=GROQ_API_KEY)

# ============================================
# Edge TTS Voice Configuration
# High-quality Microsoft Neural Voices with Indian language support
# ============================================
VOICE_MAP = {
    "English": {
        "Female": "en-IN-NeerjaNeural",   # Indian English Female
        "Male": "en-IN-PrabhatNeural"     # Indian English Male
    },
    "Hindi": {
        "Female": "hi-IN-SwaraNeural",    # Hindi Female
        "Male": "hi-IN-MadhurNeural"      # Hindi Male
    },
    "Tamil": {
        "Female": "ta-IN-PallaviNeural",  # Tamil Female
        "Male": "ta-IN-ValluvarNeural"    # Tamil Male
    },
    "Telugu": {
        "Female": "te-IN-ShrutiNeural",   # Telugu Female
        "Male": "te-IN-MohanNeural"       # Telugu Male
    },
    "Bengali": {
        "Female": "bn-IN-TanishaaNeural", # Bengali Female
        "Male": "bn-IN-BashkarNeural"     # Bengali Male
    },
    "Kannada": {
        "Female": "kn-IN-SapnaNeural",    # Kannada Female
        "Male": "kn-IN-GaganNeural"       # Kannada Male
    },
    "Malayalam": {
        "Female": "ml-IN-SobhanaNeural",  # Malayalam Female
        "Male": "ml-IN-MidhunNeural"      # Malayalam Male
    },
    "Marathi": {
        "Female": "mr-IN-AarohiNeural",   # Marathi Female
        "Male": "mr-IN-ManoharNeural"     # Marathi Male
    },
    "Gujarati": {
        "Female": "gu-IN-DhwaniNeural",   # Gujarati Female
        "Male": "gu-IN-NiranjanNeural"    # Gujarati Male
    },
    "Spanish": {
        "Female": "es-MX-DaliaNeural",
        "Male": "es-MX-JorgeNeural"
    }
}

# Helper function to get available Groq models
def get_available_groq_models():
    return [
        "llama-3.1-8b-instant",
        "llama-3.2-3b-instruct",
        "llama-3.2-11b-vision-preview",
        "llama-3.3-70b-versatile",
        "mixtral-8x7b-32768",
        "gemma-7b-it"
    ]

print("✓ Edge TTS available (Microsoft Neural Voices)")
print()
print("✓ Available Indian TTS Voices:")
for lang in ["English", "Hindi", "Tamil", "Telugu"]:
    f_voice = VOICE_MAP[lang]["Female"].split("-")[-1].replace("Neural", "")
    m_voice = VOICE_MAP[lang]["Male"].split("-")[-1].replace("Neural", "")
    print(f"  - {lang}: {f_voice} (F), {m_voice} (M)")


✓ pydub + ffmpeg available for high-quality audio combining
✓ Edge TTS available (Microsoft Neural Voices)

✓ Available Indian TTS Voices:
  - English: Neerja (F), Prabhat (M)
  - Hindi: Swara (F), Madhur (M)
  - Tamil: Pallavi (F), Valluvar (M)
  - Telugu: Shruti (F), Mohan (M)


In [9]:
print("\nTesting Groq API connection by listing available models...")
try:
    models = groq_client.models.list()
    print("\n✓ Groq API connection successful!")
    print("Available models:")
    for model in models.data:
        print(f"  - {model.id}")
except Exception as e:
    print(f"\n❌ Groq API connection failed: {e}")
    print("Please check your GROQ_API_KEY in Colab secrets or environment variables.")



Testing Groq API connection by listing available models...

✓ Groq API connection successful!
Available models:
  - whisper-large-v3
  - openai/gpt-oss-20b
  - openai/gpt-oss-safeguard-20b
  - meta-llama/llama-4-scout-17b-16e-instruct
  - moonshotai/kimi-k2-instruct-0905
  - moonshotai/kimi-k2-instruct
  - groq/compound
  - meta-llama/llama-guard-4-12b
  - llama-3.1-8b-instant
  - canopylabs/orpheus-v1-english
  - qwen/qwen3-32b
  - groq/compound-mini
  - whisper-large-v3-turbo
  - llama-3.3-70b-versatile
  - meta-llama/llama-prompt-guard-2-86m
  - canopylabs/orpheus-arabic-saudi
  - meta-llama/llama-prompt-guard-2-22m
  - meta-llama/llama-4-maverick-17b-128e-instruct
  - allam-2-7b
  - openai/gpt-oss-120b


## Step 3: Content Processing Functions

In [10]:
# ============================================
# Content Processing Functions
# ============================================

import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urlparse, unquote

def get_wikipedia_content(article_input):
    """Fetch and summarize Wikipedia article

    Accepts either:
    - Article title: "Polar vortex"
    - Wikipedia URL: "https://en.wikipedia.org/wiki/Polar_vortex"
    """
    try:
        # Check if input is a URL
        if article_input.startswith('http') or 'wikipedia.org' in article_input:
            # Extract article title from URL
            # URL format: https://en.wikipedia.org/wiki/Article_Name
            parsed = urlparse(article_input)
            path = parsed.path

            # Extract the article name from /wiki/Article_Name
            if '/wiki/' in path:
                article_title = path.split('/wiki/')[-1]
                # Decode URL encoding (e.g., %20 -> space)
                article_title = unquote(article_title)
                # Replace underscores with spaces
                article_title = article_title.replace('_', ' ')
            else:
                article_title = article_input

            print(f"🔍 Extracted article title: {article_title}")
        else:
            article_title = article_input

        print(f"🔍 Searching Wikipedia for: {article_title}")

        # Search Wikipedia
        page = wikipedia.page(article_title, auto_suggest=False)
        summary = wikipedia.summary(article_title, sentences=10, auto_suggest=False)

        print(f"✓ Found: {page.title}")

        return {
            "content": f"# {page.title}\n\n{summary}\n\n## Full Article\n\n{page.content[:3000]}...",
            "source": f"Wikipedia: {page.title}",
            "title": page.title,
            "url": page.url
        }
    except wikipedia.exceptions.PageError:
        print(f"⚠ Page not found. Trying search...")
        try:
            # Try searching instead
            search_results = wikipedia.search(article_title if 'article_title' in dir() else article_input)
            if search_results:
                print(f"  Found similar: {search_results[:3]}")
                # Try the first result
                page = wikipedia.page(search_results[0], auto_suggest=False)
                summary = wikipedia.summary(search_results[0], sentences=10, auto_suggest=False)
                print(f"✓ Using: {page.title}")
                return {
                    "content": f"# {page.title}\n\n{summary}\n\n## Full Article\n\n{page.content[:3000]}...",
                    "source": f"Wikipedia: {page.title}",
                    "title": page.title,
                    "url": page.url
                }
        except Exception as e:
            pass
        return {"error": f"Article not found: {article_input}"}
    except wikipedia.exceptions.DisambiguationError as e:
        print(f"⚠ Multiple results found:")
        for opt in e.options[:5]:
            print(f"  - {opt}")
        return {"error": f"Multiple matches. Try one of these: {e.options[:5]}"}
    except Exception as e:
        return {"error": str(e)}


def fetch_url_content(url):
    """Fetch and extract text content from a URL"""
    try:
        print(f"🌐 Fetching content from: {url}")

        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        response = requests.get(url, headers=headers, timeout=15)
        response.raise_for_status()

        # Parse HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Remove script and style elements
        for script in soup(["script", "style", "nav", "footer", "header", "aside"]):
            script.decompose()

        # Get title
        title = soup.title.string if soup.title else url

        # Extract main content
        main_content = None
        for selector in ['article', 'main', '.content', '.post-content', '#content', '.article-body', '#mw-content-text']:
            main_content = soup.select_one(selector)
            if main_content:
                break

        if main_content:
            text = main_content.get_text(separator='\n', strip=True)
        else:
            text = soup.get_text(separator='\n', strip=True)

        # Clean up text
        lines = [line.strip() for line in text.split('\n') if line.strip()]
        text = '\n'.join(lines)

        if len(text) > 5000:
            text = text[:5000] + "..."

        print(f"✓ Extracted {len(text)} characters from: {title[:50]}")

        return {
            "content": f"# {title}\n\n{text}",
            "source": f"URL: {url}",
            "title": title,
            "url": url
        }
    except Exception as e:
        return {"error": f"Failed to fetch URL: {str(e)}"}


def search_online(query):
    """Search online and generate comprehensive content using Groq"""
    print(f"🔍 Searching online for: {query}")

    prompt = f"""You are a research assistant. Provide a comprehensive, well-researched article about: {query}

Include:
1. Overview and introduction
2. Key concepts and explanations
3. Important facts and details
4. Real-world examples or applications
5. Current state and future trends

Write in a clear, informative style suitable for a podcast discussion.
Length: 800-1200 words."""

    models_to_try = [
        "llama-3.1-8b-instant",
        "llama-3.3-70b-versatile",
        "llama-3.2-11b-vision-preview",
        "mixtral-8x7b-32768"
    ]

    response = None
    for model_name in models_to_try:
        try:
            response = groq_client.chat.completions.create(
                model=model_name,
                messages=[
                    {"role": "system", "content": "You are a knowledgeable research assistant."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.7,
                max_tokens=2500
            )
            print(f"✓ Research complete using: {model_name}")
            break
        except Exception as e:
            print(f"⚠ {model_name} failed, trying next...")
            continue

    if response is None:
        return {"error": "All AI models failed. Please try again later."}

    content = response.choices[0].message.content
    return {
        "content": f"# {query}\n\n{content}",
        "source": f"AI Research: {query}",
        "title": query
    }


def upload_content(file_path=None):
    """Upload and read file content"""
    if IN_COLAB:
        print("📤 Upload your file...")
        uploaded = files.upload()
        for filename, content in uploaded.items():
            try:
                text_content = content.decode('utf-8')
            except:
                text_content = content.decode('latin-1')
            print(f"✓ Uploaded: {filename} ({len(text_content)} characters)")
            return {
                "content": text_content,
                "source": f"Uploaded: {filename}",
                "title": filename
            }
        return {"error": "No file uploaded"}
    else:
        if file_path is None:
            file_path = input("📁 Enter file path: ")
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                text_content = f.read()
            print(f"✓ Loaded: {file_path} ({len(text_content)} characters)")
            return {
                "content": text_content,
                "source": f"File: {file_path}",
                "title": os.path.basename(file_path)
            }
        except Exception as e:
            return {"error": f"Error reading file: {str(e)}"}


def summarize_content(content, max_words=500):
    """Summarize content using AI for a quick preview"""
    print("\n📝 Generating summary...")

    prompt = f"""Summarize the following content in a clear, concise way.
Highlight the main topics, key points, and interesting facts.
Keep it under {max_words} words.

Content:
{content[:4000]}

Summary:"""

    try:
        response = groq_client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=[
                {"role": "system", "content": "You are a helpful summarizer."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.5,
            max_tokens=600
        )
        return response.choices[0].message.content
    except Exception as e:
        words = content.split()[:max_words]
        return ' '.join(words) + "..."



## Step 4: Script Generation Function

In [11]:
def generate_script(content, lang1="English", lang2="Hindi"):
    """Generate natural conversation script with TWO distinct named characters

    Characters:
    - P1 = Priya (Female host)
    - P2 = Arjun (Male co-host)
    """

    prompt = f"""Create a professional podcast conversation between exactly TWO hosts discussing:

{content[:2000]}

=== MANDATORY CHARACTER RULES ===

You MUST use exactly these TWO characters:
- PRIYA (Female) - The main host who introduces topics and asks questions
- ARJUN (Male) - The co-host who provides explanations and insights

The conversation MUST alternate between Priya and Arjun:
- Line 1: Priya speaks
- Line 2: Arjun speaks
- Line 3: Priya speaks
- Line 4: Arjun speaks
- And so on...

=== LANGUAGE MIX ===

74% ENGLISH - Main content and explanations
20% HINDI (Roman script) - Professional expressions like:
- "Bilkul sahi" (Absolutely right)
- "Bahut important hai" (Very important)
- "Yeh interesting point hai" (This is an interesting point)
3% CONVERSATIONAL FILLERS - Natural reactions that make conversation feel real:
- "Hmmm..." (thoughtful acknowledgment)
- "Acccha..." (understanding/realization)
- "Is that so?" (curiosity)
- "Ohh, I see!" (comprehension)
- "Ohh..." (realization)
3% FORMAL PAUSES - "Indeed", "Well", "Precisely", "Absolutely"

=== CONVERSATION GUIDELINES ===

1. Priya starts with a welcome and introduces the topic
2. Arjun provides the first explanation
3. They alternate naturally, each contributing unique insights
4. Keep it professional and dignified
5. End with a proper conclusion (8-10 exchanges total)
6. Use conversational fillers naturally: "Hmmm...", "Acccha...", "Is that so?", "Ohh, I see!", "Ohh..."
7. Fillers should come at the START of responses to show active listening
8. NO slang: avoid "Umm", "Jai ho", "Kya baat", "Yaar"

=== EXAMPLE FORMAT ===

Priya: "Welcome to our podcast. Today we're discussing [topic]. Arjun, would you like to start us off?"
Arjun: "Absolutely, Priya. This topic is fascinating because..."
Priya: "Hmmm... that's an excellent point. Aur ek important aspect yeh hai ki..."
Arjun: "Acccha, I see what you mean. And if we consider..."
Priya: "Ohh, I see! That makes sense. Bilkul sahi."
Arjun: "Is that so? Well, there's actually more to it..."
Priya: "Indeed. Well, that covers our main points. Thank you, Arjun."
Arjun: "Thank you, Priya. It was a great discussion."

=== OUTPUT FORMAT (CRITICAL) ===

Return ONLY valid JSON with "speaker" as P1 or P2:
- P1 = Priya (Female)
- P2 = Arjun (Male)

{{"script": [
  {{"speaker": "P1", "text": "Welcome to our podcast..."}},
  {{"speaker": "P2", "text": "Thank you, Priya..."}},
  {{"speaker": "P1", "text": "..."}},
  {{"speaker": "P2", "text": "..."}}
]}}

IMPORTANT:
- Use P1 for Priya, P2 for Arjun in the "speaker" field
- Include the character names naturally in the dialogue text
- Ensure P1 and P2 alternate properly"""

    models_to_try = [
        "llama-3.1-8b-instant",
        "llama-3.3-70b-versatile",
        "llama-3.2-11b-vision-preview",
        "mixtral-8x7b-32768"
    ]

    response = None
    last_error = None

    print("🎬 Generating podcast script...")
    print("   Characters: Priya (Female) & Arjun (Male)")
    print("   Language: 75% English | 20% Hindi | 5% Formal pauses")
    print()

    for i, model_name in enumerate(models_to_try, 1):
        try:
            print(f"[{i}/{len(models_to_try)}] Trying model: {model_name}...")
            response = groq_client.chat.completions.create(
                model=model_name,
                messages=[
                    {"role": "system", "content": "You are a professional podcast scriptwriter. Create conversations between exactly two hosts: Priya (female, P1) and Arjun (male, P2). They must alternate speaking. Use P1/P2 as speaker identifiers but include names in dialogue. Respond with valid JSON only."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.7,
                max_tokens=3000,
                response_format={"type": "json_object"}
            )
            print(f"✓ Success! Using model: {model_name}")
            break
        except Exception as e:
            last_error = e
            print(f"  ✗ Error, trying next...")
            continue

    if response is None:
        raise Exception(f"❌ All models failed. Last error: {str(last_error)[:200]}")

    result = json.loads(response.choices[0].message.content)

    # Extract script array
    if "script" in result:
        script = result["script"]
    elif isinstance(result, list):
        script = result
    else:
        script = result.get("conversation", result.get("dialog", []))

    # Validate and fix alternation if needed
    fixed_script = []
    for i, line in enumerate(script):
        speaker = line.get("speaker", "P1" if i % 2 == 0 else "P2")
        # Ensure proper alternation
        expected_speaker = "P1" if i % 2 == 0 else "P2"
        fixed_script.append({
            "speaker": expected_speaker,
            "text": line.get("text", "")
        })

    return fixed_script



In [12]:
# ============================================
# Audio Generation Functions using Edge TTS
# ============================================

import warnings
warnings.filterwarnings("ignore", message="Couldn't find ffmpeg")
warnings.filterwarnings("ignore", message="Couldn't find ffprobe")

# Common Indian names for gender detection
FEMALE_NAMES = {
    'priya', 'ananya', 'neha', 'aisha', 'riya', 'pooja', 'shreya', 'kavya',
    'meera', 'divya', 'anjali', 'sneha', 'nisha', 'deepa', 'sunita', 'rekha',
    'lakshmi', 'sarita', 'geeta', 'radha', 'sita', 'uma', 'asha', 'lata',
    'swati', 'preeti', 'mamta', 'seema', 'kiran', 'jyoti', 'shanti', 'padma',
    'rani', 'suman', 'vidya', 'nandini', 'pallavi', 'rashmi', 'shweta', 'archana'
}

MALE_NAMES = {
    'arjun', 'rohan', 'vikram', 'raj', 'amit', 'suresh', 'ramesh', 'mahesh',
    'anil', 'vijay', 'sanjay', 'ajay', 'rahul', 'deepak', 'sunil', 'manoj',
    'rajesh', 'mukesh', 'dinesh', 'ganesh', 'naresh', 'rakesh', 'satish',
    'ashok', 'vinod', 'pramod', 'arun', 'varun', 'karan', 'nikhil', 'sachin',
    'mohit', 'rohit', 'sumit', 'ankit', 'vivek', 'gaurav', 'harsh', 'dev'
}

def detect_gender_from_name(name):
    """Detect gender from speaker name"""
    name_lower = name.lower().strip()

    # Check against known names
    if name_lower in FEMALE_NAMES:
        return "Female"
    elif name_lower in MALE_NAMES:
        return "Male"

    # Heuristics for Indian names
    # Names ending in 'a', 'i', 'ee' are often female
    if name_lower.endswith(('a', 'i', 'ee', 'ya', 'ti', 'ni', 'ri', 'li', 'vi', 'wi')):
        return "Female"
    # Names ending in consonants or 'n', 'sh', 'j', 'v', 'k' are often male
    elif name_lower.endswith(('n', 'sh', 'j', 'v', 'k', 'm', 'p', 't', 'r', 'l', 'y')):
        return "Male"

    # Default based on P1/P2 convention
    if name == "P1":
        return "Female"
    elif name == "P2":
        return "Male"

    return "Female"  # Default


async def _generate_edge_tts_audio(text, speaker, language, gender, index):
    """Generate audio using Edge TTS (Microsoft Neural Voices)"""
    try:
        # Get voice from map based on actual gender
        voice = VOICE_MAP.get(language, VOICE_MAP["English"]).get(gender, "en-IN-NeerjaNeural")

        print(f"  Generating audio for {speaker} using Edge TTS ({language}, {gender})...")
        print(f"    Voice: {voice}")

        # Create communicate object
        communicate = edge_tts.Communicate(text, voice)

        # Generate audio file
        audio_filename = f"audio_{index}_{speaker}.mp3"
        await communicate.save(audio_filename)

        print(f"  ✓ Generated {audio_filename}")

        return {
            "speaker": speaker,
            "text": text,
            "filename": audio_filename,
            "method": "edge_tts",
            "voice": voice,
            "gender": gender
        }
    except Exception as e:
        raise Exception(f"Edge TTS error: {str(e)}")


def generate_audio(script, p1_language="English", p1_gender="Female", p2_language="Hindi", p2_gender="Male"):
    """Generate audio from script using Edge TTS

    Automatically detects gender from speaker names in the script.
    Falls back to p1_gender/p2_gender for P1/P2 speakers.
    """
    audio_files = []

    print(f"\n🎙️ Generating audio for {len(script)} segments...")

    # First pass: detect speakers and their genders
    speakers_detected = {}
    for line in script:
        speaker = line.get("speaker", "P1")
        if speaker not in speakers_detected:
            detected_gender = detect_gender_from_name(speaker)
            speakers_detected[speaker] = detected_gender

    print(f"  Detected speakers:")
    for speaker, gender in speakers_detected.items():
        print(f"    - {speaker}: {gender}")
    print()

    async def generate_all():
        for i, line in enumerate(script):
            speaker = line.get("speaker", "P1")
            text = line.get("text", "")

            if not text.strip():
                continue

            # Determine gender from speaker name
            gender = speakers_detected.get(speaker, "Female")

            # Determine language based on gender (Female=English, Male=Hindi for variety)
            # Or use the same language with different voices
            if gender == "Female":
                language = p1_language
            else:
                language = p2_language

            try:
                audio_info = await _generate_edge_tts_audio(text, speaker, language, gender, i)
                audio_files.append(audio_info)
            except Exception as e:
                print(f"  ⚠ Error generating audio for segment {i}: {str(e)}")

    # Run async function
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            loop.run_until_complete(generate_all())
        else:
            asyncio.run(generate_all())
    except RuntimeError:
        asyncio.run(generate_all())

    return audio_files


def combine_audio_files(audio_files, output_filename="combined_podcast.mp3", silence_duration=500):
    """Combine multiple audio files into a single podcast file"""
    print(f"\n🔗 Combining {len(audio_files)} audio segments...")

    def strip_id3_tags(data):
        """Strip ID3v2 header and ID3v1 footer from MP3 data"""
        start = 0
        end = len(data)

        if data[:3] == b'ID3':
            if len(data) >= 10:
                size_bytes = data[6:10]
                size = ((size_bytes[0] & 0x7f) << 21) | \
                       ((size_bytes[1] & 0x7f) << 14) | \
                       ((size_bytes[2] & 0x7f) << 7) | \
                       (size_bytes[3] & 0x7f)
                start = 10 + size

        if len(data) >= 128 and data[-128:-125] == b'TAG':
            end = len(data) - 128

        return data[start:end]

    with open(output_filename, 'wb') as outfile:
        for i, audio_info in enumerate(audio_files):
            filename = audio_info.get("filename", "")
            if filename and os.path.exists(filename):
                try:
                    with open(filename, 'rb') as infile:
                        data = infile.read()
                        clean_data = strip_id3_tags(data)
                        outfile.write(clean_data)
                        file_size = len(clean_data) / 1024
                        speaker = audio_info.get("speaker", "?")
                        gender = audio_info.get("gender", "?")
                        print(f"  ✓ Added {filename} - {speaker} ({gender}, {file_size:.1f} KB)")
                except Exception as e:
                    print(f"  ⚠ Error reading {filename}: {e}")
            else:
                print(f"  ⚠ Skipping missing file: {filename}")

    file_size = os.path.getsize(output_filename) / (1024 * 1024)
    print(f"\n✓ Combined podcast saved: {output_filename}")
    print(f"  File size: {file_size:.2f} MB")

    return output_filename


def play_audio(filename):
    """Play audio file in Jupyter/Colab notebook"""
    if AUDIO_DISPLAY_AVAILABLE and os.path.exists(filename):
        print(f"\n🎧 Playing: {filename}")
        display(Audio(filename=filename))
    else:
        print(f"\n⚠ Audio playback not available")
        print(f"  File saved at: {os.path.abspath(filename)}")



## Step 6: Main Workflow - Example Usage

In [13]:
# ============================================
# 📥 STEP 1: GET YOUR CONTENT
# ============================================

import sys

# Display menu FIRST (with flush to ensure it shows before input)
print("=" * 60)
print("🎙️ PODCAST CONTENT SOURCE")
print("=" * 60)
print()
print("How would you like to provide content for your podcast?")
print()
print("  1️⃣  Search Online  - AI-powered research on any topic")
print("  2️⃣  Wikipedia      - Fetch article from Wikipedia")
print("  3️⃣  URL/Link       - Extract content from a webpage")
print("  4️⃣  Upload File    - Upload a text/document file")
print("  5️⃣  Paste Text     - Copy and paste your own content")
print()
print("=" * 60)
sys.stdout.flush()  # Force output to display before input

choice = input("\n👉 Enter your choice (1-5): ").strip()

content_data = None

if choice == "1":
    print("\n" + "=" * 60)
    print("🔍 ONLINE SEARCH")
    print("=" * 60)
    sys.stdout.flush()
    query = input("\nWhat topic would you like to research?\n→ ").strip()
    if query:
        content_data = search_online(query)
    else:
        print("⚠ No query provided")

elif choice == "2":
    print("\n" + "=" * 60)
    print("📚 WIKIPEDIA SEARCH")
    print("=" * 60)
    sys.stdout.flush()
    article = input("\nEnter Wikipedia article title:\n→ ").strip()
    if article:
        content_data = get_wikipedia_content(article)
    else:
        print("⚠ No article title provided")

elif choice == "3":
    print("\n" + "=" * 60)
    print("🌐 FETCH FROM URL")
    print("=" * 60)
    sys.stdout.flush()
    url = input("\nEnter the webpage URL:\n→ ").strip()
    if url:
        if not url.startswith('http'):
            url = 'https://' + url
        content_data = fetch_url_content(url)
    else:
        print("⚠ No URL provided")

elif choice == "4":
    print("\n" + "=" * 60)
    print("📤 FILE UPLOAD")
    print("=" * 60)
    sys.stdout.flush()
    content_data = upload_content()

elif choice == "5":
    print("\n" + "=" * 60)
    print("📋 PASTE YOUR CONTENT")
    print("=" * 60)
    print("\nPaste your content below.")
    print("When done, press Enter on an empty line twice.")
    print("-" * 40)
    sys.stdout.flush()

    lines = []
    empty_count = 0
    while True:
        try:
            line = input()
            if line == "":
                empty_count += 1
                if empty_count >= 2:
                    break
            else:
                empty_count = 0
            lines.append(line)
        except EOFError:
            break

    pasted_text = "\n".join(lines).strip()
    if pasted_text:
        title = input("\nGive your content a title: ").strip() or "Pasted Content"
        content_data = {
            "content": pasted_text,
            "source": "Pasted by user",
            "title": title
        }
        print(f"✓ Received {len(pasted_text)} characters")
    else:
        print("⚠ No content pasted")

else:
    print("\n⚠ Invalid choice. Please enter 1, 2, 3, 4, or 5")
    print("   Run this cell again to try.")

# Display results and summary
if content_data and "error" not in content_data:
    print("\n" + "=" * 60)
    print("✅ CONTENT RETRIEVED SUCCESSFULLY")
    print("=" * 60)
    print(f"\n📌 Title: {content_data.get('title', 'N/A')}")
    print(f"📍 Source: {content_data.get('source', 'N/A')}")
    print(f"📊 Length: {len(content_data.get('content', ''))} characters")

    # Generate and display summary
    summary = summarize_content(content_data.get('content', ''))

    print("\n" + "-" * 60)
    print("📝 CONTENT SUMMARY")
    print("-" * 60)
    print(summary)
    print("-" * 60)

    print("\n✅ Content is ready! Proceed to the next cell to generate your podcast script.")

elif content_data and "error" in content_data:
    print(f"\n❌ Error: {content_data['error']}")
    print("Please run this cell again with different input.")



🎙️ PODCAST CONTENT SOURCE

How would you like to provide content for your podcast?

  1️⃣  Search Online  - AI-powered research on any topic
  2️⃣  Wikipedia      - Fetch article from Wikipedia
  3️⃣  URL/Link       - Extract content from a webpage
  4️⃣  Upload File    - Upload a text/document file
  5️⃣  Paste Text     - Copy and paste your own content


👉 Enter your choice (1-5): 3

🌐 FETCH FROM URL

Enter the webpage URL:
→ https://en.wikipedia.org/wiki/List_of_wars_involving_Poland
🌐 Fetching content from: https://en.wikipedia.org/wiki/List_of_wars_involving_Poland
✓ Extracted 5003 characters from: List of wars involving Poland - Wikipedia

✅ CONTENT RETRIEVED SUCCESSFULLY

📌 Title: List of wars involving Poland - Wikipedia
📍 Source: URL: https://en.wikipedia.org/wiki/List_of_wars_involving_Poland
📊 Length: 5048 characters

📝 Generating summary...

------------------------------------------------------------
📝 CONTENT SUMMARY
-------------------------------------------------------

In [14]:
# Step 2: Generate Script
# Characters: Priya (Female) & Arjun (Male)

script = generate_script(
    content=content_data.get("content", "")
)

print("\n" + "=" * 60)
print("📜 GENERATED PODCAST SCRIPT")
print("=" * 60)
print("Characters: Priya (👩 Female) & Arjun (👨 Male)")
print("Language: 75% English | 20% Hindi\n")

for line in script:
    speaker_id = line.get('speaker', 'P1')
    text = line.get('text', '')

    # Map P1 → Priya (Female), P2 → Arjun (Male)
    if speaker_id == "P1":
        emoji = "👩"
        name = "Priya"
    else:
        emoji = "👨"
        name = "Arjun"

    print(f"{emoji} {name}: {text}")
    print()

print("=" * 60)
print(f"✅ Script generated with {len(script)} dialogue segments")
print("   👩 Priya (Female) - English voice")
print("   👨 Arjun (Male) - Hindi voice")
print("\n→ Proceed to the next cell to generate audio")



🎬 Generating podcast script...
   Characters: Priya (Female) & Arjun (Male)
   Language: 75% English | 20% Hindi | 5% Formal pauses

[1/4] Trying model: llama-3.1-8b-instant...
✓ Success! Using model: llama-3.1-8b-instant

📜 GENERATED PODCAST SCRIPT
Characters: Priya (👩 Female) & Arjun (👨 Male)
Language: 75% English | 20% Hindi

👩 Priya: Welcome to our podcast. Today, we're discussing the List of Wars Involving Poland, as mentioned on Wikipedia.

👨 Arjun: Yes, Priya, this topic is fascinating because it gives us a chronological list of wars in which Poland or its predecessor states took an active part, extending from the reign of Mieszko I to the present.

👩 Priya: Hmmm... that's an interesting point. I was wondering, Arjun, what's the significance of Mieszko I in this context?

👨 Arjun: Acccha, I see what you mean. Mieszko I was a powerful duke who united various West Slavic tribes and established the Polish state, which is why he's considered a crucial figure in Polish history.

👩 Pr

In [15]:
# Step 3: Generate Audio & Create Podcast
# Priya → Female English voice | Arjun → Male Hindi voice

try:
    audio_files = generate_audio(
        script=script,
        p1_language="English",   # Priya speaks English
        p1_gender="Female",      # Priya is female
        p2_language="English",   # Arjun also speaks English (with Hindi accent)
        p2_gender="Male"         # Arjun is male
    )

    print(f"\n✓ Generated {len(audio_files)} audio segments")
    print("\nAudio files created:")
    for audio in audio_files:
        gender = audio.get('gender', '?')
        voice = audio.get('voice', 'unknown')
        name = "Priya" if gender == "Female" else "Arjun"
        print(f"  👩 {name}: {audio['filename']}" if gender == "Female" else f"  👨 {name}: {audio['filename']}")

    # Combine all audio segments
    combined_file = combine_audio_files(
        audio_files=audio_files,
        output_filename="combined_podcast.mp3"
    )

    # Play the combined podcast
    print("\n" + "=" * 60)
    print("🎧 PODCAST PLAYER")
    print("=" * 60)
    play_audio(combined_file)

    if IN_COLAB:
        from google.colab import files
        print("\n📥 Click below to download:")
        files.download(combined_file)

except Exception as e:
    print(f"\n⚠ Audio generation failed: {e}")
    print("\nTroubleshooting:")
    print("1. Re-run Cell 4 (imports)")
    print("2. Re-run Cell 9 (audio functions)")
    print("3. Try again")




🎙️ Generating audio for 10 segments...
  Detected speakers:
    - P1: Female
    - P2: Male

  Generating audio for P1 using Edge TTS (English, Female)...
    Voice: en-IN-NeerjaNeural
  ✓ Generated audio_0_P1.mp3
  Generating audio for P2 using Edge TTS (English, Male)...
    Voice: en-IN-PrabhatNeural
  ✓ Generated audio_1_P2.mp3
  Generating audio for P1 using Edge TTS (English, Female)...
    Voice: en-IN-NeerjaNeural
  ✓ Generated audio_2_P1.mp3
  Generating audio for P2 using Edge TTS (English, Male)...
    Voice: en-IN-PrabhatNeural
  ✓ Generated audio_3_P2.mp3
  Generating audio for P1 using Edge TTS (English, Female)...
    Voice: en-IN-NeerjaNeural
  ✓ Generated audio_4_P1.mp3
  Generating audio for P2 using Edge TTS (English, Male)...
    Voice: en-IN-PrabhatNeural
  ✓ Generated audio_5_P2.mp3
  Generating audio for P1 using Edge TTS (English, Female)...
    Voice: en-IN-NeerjaNeural
  ✓ Generated audio_6_P1.mp3
  Generating audio for P2 using Edge TTS (English, Male)...
  


📥 Click below to download:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>